In [1]:
#%matplotlib inline 
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import math
from datetime import datetime as dt
from datetime import timedelta

from sklearn.svm import SVR 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Activation, Dropout, Dropout, Dense, Flatten
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

In [2]:
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/Data/Bike_Sharing/data_final.csv')
data = data.drop(data.columns[0], axis =1)
data['timestamp'] = data['timestamp'].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S.%f')

weather = pd.read_csv('/content/drive/MyDrive/Data/Bike_Sharing/weather_data.csv')
weather = weather.drop(weather.columns[0], axis =1)
weather['timestamp'] = weather['timestamp'].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S.%f')

Mounted at /content/drive


In [3]:
weather_data = weather.iloc[:,1:]

In [4]:
all_stations_demand = data.set_index('timestamp')
all_stations_demand

,A.I. Virtasen aukio,"Aalto-yliopisto (M), Korkeakouluaukio","Aalto-yliopisto (M), Tietotie",Abraham Wetterin tie,Agnetankuja,Agronominkatu,Ahertajantie,Alakiventie,Albertinkatu,Annankatu,...,Viikin normaalikoulu,Viikin tiedepuisto,Viiskulma,Vilhonvuorenkatu,Voikukantie,Von Daehnin katu,Westendinasema,Westendintie,Yhdyskunnankuja,Ympyrätalo
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-05-02 15:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2016-05-02 16:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2016-05-02 17:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2016-05-02 18:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2016-05-02 19:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-31 20:00:00,0,2,0,0,0,0,0,0,1,0,...,0,1,4,0,0,0,1,0,0,2
2020-10-31 21:00:00,0,1,1,0,1,0,0,0,1,0,...,0,0,0,3,0,0,0,0,0,2
2020-10-31 22:00:00,0,3,2,0,0,1,0,0,0,5,...,0,1,1,1,0,0,0,0,0,3


In [5]:
# take sample data to test model
sample_demand = all_stations_demand.iloc[:,:200]
sample_weather = weather.iloc[:,1:]

In [6]:
sample_demand

,A.I. Virtasen aukio,"Aalto-yliopisto (M), Korkeakouluaukio","Aalto-yliopisto (M), Tietotie",Abraham Wetterin tie,Agnetankuja,Agronominkatu,Ahertajantie,Alakiventie,Albertinkatu,Annankatu,...,Munkkiniemen aukio,Muotoilijankatu,Mustikkamaa,Muurarinkuja,Muusantori,Myllypuro (M),Mäkelänkatu,Mäkitorpantie,Mäkkylän asema,Mäntyviita
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-05-02 15:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-05-02 16:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-05-02 17:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-05-02 18:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-05-02 19:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-31 20:00:00,0,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,2,0,0,0,0
2020-10-31 21:00:00,0,1,1,0,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2020-10-31 22:00:00,0,3,2,0,0,1,0,0,0,5,...,0,0,0,0,0,0,0,0,0,1


In [7]:
sample_weather

,temp,dew,humidity,precip,windspeed,cloudcover,visibility,solarenergy
0,15.3,0.4,36.28,0.0,9.2,0.1,29.4,2.100000
1,14.8,-0.1,36.07,0.0,10.3,0.1,28.2,1.800000
2,15.2,-0.3,34.75,0.0,8.8,0.0,29.7,1.500000
3,15.7,-3.1,27.35,0.0,12.2,0.0,32.9,1.100000
4,15.3,-2.1,30.12,0.0,10.4,0.0,34.7,0.600000
...,...,...,...,...,...,...,...,...
39437,7.9,7.0,93.86,0.0,10.0,90.3,15.4,0.735487
39438,8.2,7.3,94.13,0.0,6.9,91.9,15.2,0.735487
39439,8.6,7.5,92.64,0.0,7.4,91.9,17.4,0.735487
39440,8.9,6.9,87.30,0.0,7.9,91.9,26.8,0.735487


In [8]:
# create future forecast time
def create_prediction_time(start, hours):
    
    start = pd.to_datetime(start,format='%Y-%m-%d %H:%M:%S.%f')
    time_array = []
    time = start
    for i in range(hours):
        time_array.append(time)
        time = time + timedelta(hours=1)
    forecast_time = pd.DataFrame(index=time_array)
    return forecast_time

# get values, station name and drop null values
def get_value_name(all_station_temp,i):
    station_value = all_station_temp[[all_station_temp.columns[i]]]
    station_name = all_station_temp.columns[i]
    return station_value, station_name

# rewrite funxtion to have array
def preparedata_for_model(name, demand_at_station, lag, weather_data, weather_forecast_hours, split_ratio):
    
    demand_features = demand_at_station.values.reshape(-1)
    weather = weather_data.values

    # Create the features from previous demand
    for i in range(1,lag+1):
        demand_features =np.append(demand_features, np.roll(demand_at_station,i))
    demand_features = demand_features.reshape(lag+1,-1).T

    # Creating features from predicting weather data
    for j in range(1, weather_forecast_hours+1):   # predict 2 hours but get the forecast weather as 3 next hours
        weather = np.hstack((weather, np.roll(weather_data,-j, axis=0)))

    # add the weather data of previous t: because it will need time to make decision
    weather = np.hstack((np.roll(weather_data,1, axis=0), weather))   
    
    X = np.hstack((demand_features, weather))

    y = []
    for s in range (0,len(X)-2):  # minus predicting hours
          two_hours_demand = X[s+1,0] + X[s+2,0]
          y = np.append(y, two_hours_demand)

    new_y = y[lag:-(weather_forecast_hours -1)].reshape(-1,1)
    new_X = X[lag:-(weather_forecast_hours +1),:]

    total_nrow = int(new_X.shape[0])

    for i in range(total_nrow):
      if new_X[i, 0] !=0: 
        start_nrow = i
        break
      

    nrow = total_nrow - start_nrow
    print(name+' total samples: ',nrow)
    split_row = int(nrow* split_ratio)
    print('Training samples: ',split_row)
    print('Testing samples: ',nrow - split_row)

    X_train = new_X[:start_nrow+ split_row,:]
    X_test = new_X[start_nrow + split_row:,:]
    y_train = new_y[:start_nrow + split_row,:]
    y_test = new_y[start_nrow+split_row:,:] 
    return X_train, y_train, X_test, y_test, start_nrow+split_row

# validation result
def valid_result(model, testX, y_test_tract1_array, station_value, split_row, weather_forecast_hours, lag):    #add scaler if we need to 
    
    testPredict = model.predict(testX)
    testPredict = np.round(testPredict, decimals=0)
    
    rSquare_test = r2_score(y_test_tract1_array, testPredict)
    MAE = mean_absolute_error(y_test_tract1_array, testPredict)
    MSE = mean_squared_error(y_test_tract1_array, testPredict)

    print('R-squared is: %f', rSquare_test)
    print('MAE is: %f', MAE)
    print('MSE is: %f', MSE)

    new_test_tract1 = station_value.iloc[(split_row+lag):]       # check this: run from lag + split_row
    test_tract1_pred = new_test_tract1.iloc[:-(weather_forecast_hours+1)].copy()          # and check this: run until - (tw +1)
    test_tract1_pred['Forecast'] = testPredict
    test_tract1_pred.columns = ['Real', 'Forecast']
    
    return test_tract1_pred, rSquare_test, MAE, MSE

In [9]:
# Define CNN LSTM model
def cnn_lstm_model(units, trainX, testX, y_train_tract1_array, y_test_tract1_array):
    model = Sequential()

    model.add(Conv1D(filters=30, kernel_size=5, padding='same', activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
    model.add(MaxPooling1D(pool_size=1))

    model.add(LSTM(units,return_sequences=True,kernel_initializer='lecun_uniform'))
    model.add(Dropout(0.2))    
    model.add(LSTM(units, return_sequences=True))
    model.add(Dropout(0.2))    
    model.add(LSTM(units))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    model.fit(trainX, y_train_tract1_array, batch_size=20, epochs=20, validation_data=(testX, y_test_tract1_array), verbose=0)
    return model

In [10]:
# Input data
all_station_temp=sample_demand
weather_data=sample_weather
lag=336
weather_forecast_hours=5
predicting_hours=2

# run the model
station_name_results = list()
rSquare_results = list()
MAE_results = list()
MSE_results = list()

for i in range(len(all_station_temp.columns)):
    
    # preprocessing
    station_value, station_name = get_value_name(all_station_temp,i)

    X_train_tract1_array, y_train_tract1_array, X_test_tract1_array, y_test_tract1_array, split_row = preparedata_for_model(station_name, station_value, lag, weather_data, weather_forecast_hours, split_ratio=0.8)
    

    trainX = np.reshape(X_train_tract1_array, (X_train_tract1_array.shape[0],1,X_train_tract1_array.shape[1]))
    testX = np.reshape(X_test_tract1_array, (X_test_tract1_array.shape[0],1,X_test_tract1_array.shape[1]))                
    
    # LSTM modelling & forecast
    model = cnn_lstm_model(30, trainX, testX, y_train_tract1_array, y_test_tract1_array)             
    test_tract1_pred, rSquare_test, MAE, MSE = valid_result(model, testX, y_test_tract1_array, station_value, split_row, weather_forecast_hours, lag)        
            
    
    #Save result
    station_name_results.append(station_name)
    rSquare_results.append(rSquare_test)
    MAE_results.append(MAE)
    MSE_results.append(MSE)

    # plot result
    plt.figure(figsize=(20,5))
    plt.plot(test_tract1_pred)        
    plt.ylabel('Number of Pickup')
    plt.legend(loc='upper right')
    plt.title(station_name + '- Demand Forecast')
    plt.show() 

model.summary()
mean_rSquare =   sum(rSquare_results) / len(rSquare_results)
print("mean_rSquare: ", mean_rSquare)
#station_name_results.append('Mean of all station')
#rSquare_results.append(mean_rSquare)

results = pd.DataFrame({'station': station_name_results, 'rSquare': rSquare_results, 'MAE': MAE_results, 'MSE': MSE_results}, columns=['station', 'rSquare', 'MAE', 'MSE'])

results.to_csv('cnn_lstm_336_5_1.csv')

Output hidden; open in https://colab.research.google.com to view.